# Anime faces generator (styleGAN)

### Imports

In [1]:
import os, math
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import backend

import utils
from callbacks import UpdateVariables, SaveSamples, SaveModels
from gan import GAN

### Check GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")

if gpus:

	try:
		tf.config.experimental.set_visible_devices(gpus[0], "GPU")
		print("Using GPU :)")

	except RuntimeError as e:
		print(e)

else:
	print("Using CPU :(")

Using GPU :)


### Settings

In [3]:
DATA_DIR = "./datasets/dataset_128/"
IMAGE_SIZE = 128
NB_CHANNELS = 3

OUTPUT_DIR = "./output/"
SAMPLES_DIR = os.path.join(OUTPUT_DIR, "images/")
MODELS_DIR = os.path.join(OUTPUT_DIR, "models/")
OUTPUT_SHAPE = (7, 8)
MARGIN = IMAGE_SIZE // 8
SAVE_PER_EPOCH = 20

LATENT_DIM = 512
MAPPING_LAYERS = 8
MIN_IMAGE_SIZE = 4
MAX_FILTERS = 512
MIN_FILTERS = 64
KERNEL_SIZE = 3
ALPHA = 0.2
GAIN = 1.3

LEARNING_RATE = 0.001
MAPPING_LR_RATIO = 0.01
BETA_1 = 0.
BETA_2 = 0.99
EPSILON = 10e-8
STYLE_MIX_PROBA = 0.5
GRADIENT_PENALTY_COEF = 10.
GRADIENT_PENALTY_INTERVAL = 4
MA_HALF_LIFE = 10.

BATCH_SIZE = 8
NB_EPOCHS = 1000

### Dataset

In [4]:
dataset = preprocessing.image_dataset_from_directory(
	DATA_DIR,
	label_mode = None,
	color_mode = "rgb",
	batch_size = BATCH_SIZE,
	image_size = (IMAGE_SIZE, IMAGE_SIZE),
	shuffle = True
)

dataset = dataset.map(utils.norm_img)

NB_DATA = len(os.listdir(DATA_DIR))
NB_BATCHS = math.ceil(NB_DATA / BATCH_SIZE)

Found 30684 files belonging to 1 classes.


### Model

In [5]:
gan = GAN(
	latent_dim = LATENT_DIM,
	image_size = IMAGE_SIZE,
	nb_channels = NB_CHANNELS,
	mapping_layers = MAPPING_LAYERS,
	mapping_lr_ratio = MAPPING_LR_RATIO,
	min_image_size = MIN_IMAGE_SIZE,
	max_filters = MAX_FILTERS,
	min_filters = MIN_FILTERS,
	kernel_size = KERNEL_SIZE,
	alpha = ALPHA,
	gain = GAIN,
	style_mix_proba = STYLE_MIX_PROBA,
	gp_coef = GRADIENT_PENALTY_COEF,
	gp_interval = GRADIENT_PENALTY_INTERVAL,
	ma_beta = 0.5 ** (BATCH_SIZE / (MA_HALF_LIFE * 1000.)) if MA_HALF_LIFE > 0. else 0.,
	nb_batchs = NB_BATCHS
)

gan.compile(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

print("Mapping: {} -> {} | {} parameters".format(gan.mapping.input_shape, gan.mapping.output_shape, gan.mapping.count_params()))
print("Generator: {} -> {} | {} parameters".format(gan.generator.input_shape[0], gan.generator.output_shape, gan.generator.count_params()))
print("Discriminator: {} -> {} | {} parameters".format(gan.discriminator.input_shape, gan.discriminator.output_shape, gan.discriminator.count_params()))

Mapping: (None, 512) -> (None, 512) | 2101248 parameters
Generator: (None, 1) -> (None, 128, 128, 3) | 17157085 parameters
Discriminator: (None, 128, 128, 3) -> (None, 1) | 16438401 parameters


### First run / Continue

In [6]:
if not os.path.exists(SAMPLES_DIR):
	os.makedirs(SAMPLES_DIR)

if not os.path.exists(MODELS_DIR):
	os.makedirs(MODELS_DIR)

utils.reset_rand()
initial_epoch = gan.load_weights(MODELS_DIR)

if initial_epoch == 0:
	nb_blocks = int(math.log(IMAGE_SIZE, 2)) - int(math.log(MIN_IMAGE_SIZE, 2)) + 1
	samples_z = np.random.normal(0., 1., (OUTPUT_SHAPE[0] * OUTPUT_SHAPE[1], LATENT_DIM))
	samples_noise = np.random.normal(0., 1., ((nb_blocks * 2) - 1, OUTPUT_SHAPE[0] * OUTPUT_SHAPE[1], IMAGE_SIZE, IMAGE_SIZE, 1))
	np.save(os.path.join(OUTPUT_DIR, "samples_z.npy"), samples_z)
	np.save(os.path.join(OUTPUT_DIR, "samples_noise.npy"), samples_noise)

else:
	samples_z = np.load(os.path.join(OUTPUT_DIR, "samples_z.npy"))
	samples_noise = np.load(os.path.join(OUTPUT_DIR, "samples_noise.npy"))

### Training

In [ ]:
history = gan.fit(
	dataset,
	batch_size = BATCH_SIZE,
	epochs = NB_EPOCHS,
	shuffle = True,
	initial_epoch = initial_epoch,
	callbacks = [
		UpdateVariables(),
		SaveSamples(samples_z, samples_noise, BATCH_SIZE, MARGIN, OUTPUT_SHAPE, OUTPUT_DIR, SAMPLES_DIR, SAVE_PER_EPOCH, NB_BATCHS),
		SaveModels(MODELS_DIR)
	]
)